<a href="https://colab.research.google.com/github/sada1908/edyoda_assign/blob/main/edyoda_coderAssign_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install tiktoken

In [1]:
from langchain.agents import AgentOutputParser, BaseSingleActionAgent
from langchain_core.agents import AgentAction # Corrected import
from langchain.schema import AgentFinish
from langchain.llms import OpenAI
from typing import Any, List, Optional, Dict
import hashlib
from pydantic import Field # Import Field for explicit type hinting
from dataclasses import dataclass, field

In [2]:
import os

In [3]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
class CodeExplainerAgent(BaseSingleActionAgent):
    llm: OpenAI = Field(...)
    memory_store: Dict[str, str] = Field(default_factory=dict)

    system_prompt: str = (
        "You are a Python expert. Explain the following code as if you're teaching a beginner.\n"
        "Use simple terms. Mention what each line does, and point out loops, conditionals, or libraries.\n\n"
        "Example:\n"
        "Input:\nfor i in range(5): print(i)\n"
        "Output:\nThis loop runs 5 times. Each time, it prints the current number from 0 to 4.\n\n"
    )

    def _get_memory_key(self, code_snippet: str) -> str:
        return hashlib.md5(code_snippet.encode()).hexdigest()

    def _format_explanation(self, response: str) -> str:
        return response.strip().replace("\n", "\n\n")

    def plan(self, intermediate_steps: List, inputs: Dict[str, str]) -> AgentFinish:
        code = inputs["input"]
        key = self._get_memory_key(code)

        if key in self.memory_store:
            explanation = self.memory_store[key]
            return AgentFinish(return_values={"output": explanation}, log="Used memory.")

        prompt = self.system_prompt + f"Input:\n{code}\nOutput:"
        response = self.llm(prompt)
        explanation = self._format_explanation(response)

        self.memory_store[key] = explanation
        return AgentFinish(return_values={"output": explanation}, log="Generated explanation.")

    async def aplan(self, intermediate_steps: List, inputs: Dict[str, str]) -> AgentFinish:
        return self.plan(intermediate_steps, inputs)

    @property
    def input_keys(self) -> List[str]:
        return ["input"]

In [5]:
llm = OpenAI(temperature=0)
agent = CodeExplainerAgent.construct(llm=llm)

test_snippets = [
    "for i in range(5): print(i)",
    "if x > 0:\n    print('Positive')\nelse:\n    print('Non-positive')",
    "import pandas as pd\ndf = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})\nprint(df.head())"
]

for snippet in test_snippets:
    print("\n=== Code Snippet ===")
    print(snippet)
    print("\n💬 Explanation:")
    result = agent.plan([], {"input": snippet})
    print(result.return_values["output"])

/tmp/ipython-input-5-377671217.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/tmp/ipython-input-5-377671217.py:2: PydanticDeprecatedSince20: The `construct` method is deprecated; use `model_construct` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent = CodeExplainerAgent.construct(llm=llm)
/tmp/ipython-input-4-1404717354.py:28: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(prompt)



=== Code Snippet ===
for i in range(5): print(i)

💬 Explanation:
This code uses a loop to print out the numbers 0 to 4. The "range(5)" part tells the loop to run 5 times, and the "i" represents the current number in the loop. So each time the loop runs, it will print out the current number, starting with 0 and ending with 4.

=== Code Snippet ===
if x > 0:
    print('Positive')
else:
    print('Non-positive')

💬 Explanation:
This code checks if the value of x is greater than 0. If it is, it will print 'Positive'. If it is not, it will print 'Non-positive'. This is an example of a conditional statement, where the code will do different things depending on the value of x.

=== Code Snippet ===
import pandas as pd
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
print(df.head())

💬 Explanation:
This line imports a library called "pandas" and gives it the nickname "pd". This library is used for data analysis and manipulation.

The next line creates a new variable called "df" and assigns it a